In [1]:
import pandas as pd
import numpy as np
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister,execute,IBMQ
from qiskit import IBMQ, Aer, transpile, assemble
from qiskit.visualization import plot_histogram, plot_bloch_multivector, array_to_latex
from qiskit.extensions import Initialize
from qiskit.ignis.verification import marginal_counts
from qiskit.quantum_info import random_statevector
from math import sqrt, pi
from qiskit.visualization import array_to_latex
from qiskit.circuit.library.standard_gates import SGate

circuit=QuantumCircuit(3,1)
circuit.initialize([1/np.sqrt(2), 1/np.sqrt(2)], 1)
circuit.initialize([3/5, 4/5], 2)

circuit.h(0)


circuit.ccx(0,1,2)
circuit.ccx(0,2,1)
circuit.ccx(0,1,2)

circuit.h(0)
circuit.measure(0,0)
display(circuit.draw())

from qiskit import Aer

backend = Aer.get_backend('aer_simulator')

shots = 10000
counts = backend.run(circuit,shots=shots).result().get_counts()
probs = {string:count/shots for string,count in counts.items()}
display(probs)

p1=probs['1']
s=1-2*p1
innerProduct=sqrt(abs(s))
print("The magnitude of the inner product is ",innerProduct)


/tmp/ipykernel_1925/2714530584.py:7: DeprecationWarning: The qiskit.ignis package is deprecated and has been supersceded by the qiskit-experiments project. Refer to the migration guide: https://github.com/Qiskit/qiskit-ignis#migration-guide on how to migrate to the new project.
  from qiskit.ignis.verification import marginal_counts


┌───┐                            ┌───┐┌─┐
q_0: ─────────────┤ H ├───────────────■────■────■──┤ H ├┤M├
     ┌────────────┴───┴────────────┐  │  ┌─┴─┐  │  └───┘└╥┘
q_1: ┤ Initialize(0.70711,0.70711) ├──■──┤ X ├──■────────╫─
     └───┬─────────────────────┬───┘┌─┴─┐└─┬─┘┌─┴─┐      ║ 
q_2: ────┤ Initialize(0.6,0.8) ├────┤ X ├──■──┤ X ├──────╫─
         └─────────────────────┘    └───┘     └───┘      ║ 
c: 1/════════════════════════════════════════════════════╩═
                                                         0

{'1': 0.0102, '0': 0.9898}

The magnitude of the inner product is  0.9897474425326898


Answer 1. The CSWAP operation is implemented by 3 ccx gates that is 3 gates with 2 qubits as controls. Swap operation is performed on te 2 qubits with an additional conrol on the first qubit.The 3 gates applied are ccx gates with controls as the following:-
1. first and second qubit
2. first and third qubit
3. first and second qubit

Answer 2. The inner product is estimated by performing a partial measurement on the topmost qubit and finding out the probability of the qubit being in the $\left| 1 \right\rangle $ state. Then the inner product is estimated by finding out square root of 1- 2*(probability of the topmost qubit being in $\left|1\right\rangle $ state.
Mathematical proof:
let the initial state of the other 2 qubits be :$$p=a\left| 0 \right\rangle + b\left| 1 \right\rangle $$ and 
$$q=c\left| 0 \right\rangle + d\left| 1 \right\rangle $$ respectively
So, initial state of the 3 qubits is $$\left| 0 p q \right\rangle $$
on applying the hadamard gate the state becomes
$$ \frac{1}{\sqrt(2)}(\left| 0 p q \right\rangle+\left| 1 p q \right\rangle)$$
After the CSWAP operation the state becomes
$$ \frac{1}{\sqrt(2)}(\left| 0 p q \right\rangle+\left| 1 q p \right\rangle)$$
After the hadarmard gate,
$$ \frac{1}2(\left| 0 p q \right\rangle+\left| 1 p q \right\rangle+\left| 0 q p \right\rangle-\left| 1 q p \right\rangle)$$
on substituting the values of p and q and only considering the terms with the topmost qubit in $\left| 1 \right\rangle $ state and thus calculating the probability of the topmost qubit being in $\left|1\right\rangle $ state which comes out to be $$\frac{1}2(a^2d^2+b^2c^2-2abcd)$$ 
using the relations $$a^2+b^2=1 $$ and $$c^2+d^2=1$$  1- 2*(probability of the topmost qubit being in $\left|1\right\rangle $ state comes out to be equal to the square of the inner product and thus taking its square root gives us the inner product.

Answer 3. Circuit 

Answer 4. On measuring the 2 qubits in the Hadamard basis, the probability of finding the qubits in the state $$\left| 00 \right\rangle $$ is half of the square of the inner product. Thus, multiplyng this probability by 2 and taking its square root gives the inner product of the states of the qubits.

In [2]:
#Answer 4 code
import pandas as pd
import numpy as np
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister,execute,IBMQ
from qiskit import IBMQ, Aer, transpile, assemble
from qiskit.visualization import plot_histogram, plot_bloch_multivector, array_to_latex
from qiskit.extensions import Initialize
from qiskit.ignis.verification import marginal_counts
from qiskit.quantum_info import random_statevector
from math import sqrt, pi
from qiskit.visualization import array_to_latex


In [3]:
circuit=QuantumCircuit(2)
circuit.initialize([1/np.sqrt(2), 1/np.sqrt(2)], 0)
circuit.initialize([3/5, 4/5], 1)

In [4]:
circuit.cx(0,1)
circuit.h(0)

In [5]:
circuit.draw()

┌─────────────────────────────┐     ┌───┐
q_0: ┤ Initialize(0.70711,0.70711) ├──■──┤ H ├
     └───┬─────────────────────┬───┘┌─┴─┐└───┘
q_1: ────┤ Initialize(0.6,0.8) ├────┤ X ├─────
         └─────────────────────┘    └───┘

In [6]:
circuit.measure_all()

In [7]:
circuit.draw()

┌─────────────────────────────┐     ┌───┐ ░ ┌─┐   
   q_0: ┤ Initialize(0.70711,0.70711) ├──■──┤ H ├─░─┤M├───
        └───┬─────────────────────┬───┘┌─┴─┐└───┘ ░ └╥┘┌─┐
   q_1: ────┤ Initialize(0.6,0.8) ├────┤ X ├──────░──╫─┤M├
            └─────────────────────┘    └───┘      ░  ║ └╥┘
meas: 2/═════════════════════════════════════════════╩══╩═
                                                     0  1

In [8]:
from qiskit import Aer

backend = Aer.get_backend('aer_simulator')

shots = 1000000
counts = backend.run(circuit,shots=shots).result().get_counts()
probs = {string:count/shots for string,count in counts.items()}
display(probs)

{'11': 0.009961, '01': 0.009949, '00': 0.488986, '10': 0.491104}

In [9]:
Req=probs['00']
innerProduct=sqrt(2*Req)
print("The magnitude of the inner product is ",innerProduct)

The magnitude of the inner product is  0.9889246685162626
